In [1]:
import json
import os 
from PIL import Image
import cv2 as cv2
import numpy as np
import math
import imutils


import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
def getTranslationMatrix2d(dx, dy):
    """
    Returns a numpy affine transformation matrix for a 2D translation of
    (dx, dy)
    """
    return np.matrix([[1, 0, dx], [0, 1, dy], [0, 0, 1]])


def rotateImage(image, angle):
    """
    Rotates the given image about it's centre
    """

    image_size = (image.shape[1], image.shape[0])
    image_center = tuple(np.array(image_size) / 2)

    rot_mat = np.vstack([cv2.getRotationMatrix2D(image_center, angle, 1.0), [0, 0, 1]])
    trans_mat = np.identity(3)

    w2 = image_size[0] * 0.5
    h2 = image_size[1] * 0.5

    rot_mat_notranslate = np.matrix(rot_mat[0:2, 0:2])

    tl = (np.array([-w2, h2]) * rot_mat_notranslate).A[0]
    tr = (np.array([w2, h2]) * rot_mat_notranslate).A[0]
    bl = (np.array([-w2, -h2]) * rot_mat_notranslate).A[0]
    br = (np.array([w2, -h2]) * rot_mat_notranslate).A[0]

    x_coords = [pt[0] for pt in [tl, tr, bl, br]]
    x_pos = [x for x in x_coords if x > 0]
    x_neg = [x for x in x_coords if x < 0]

    y_coords = [pt[1] for pt in [tl, tr, bl, br]]
    y_pos = [y for y in y_coords if y > 0]
    y_neg = [y for y in y_coords if y < 0]

    right_bound = max(x_pos)
    left_bound = min(x_neg)
    top_bound = max(y_pos)
    bot_bound = min(y_neg)

    new_w = int(abs(right_bound - left_bound))
    new_h = int(abs(top_bound - bot_bound))
    new_image_size = (new_w, new_h)

    new_midx = new_w * 0.5
    new_midy = new_h * 0.5

    dx = int(new_midx - w2)
    dy = int(new_midy - h2)

    trans_mat = getTranslationMatrix2d(dx, dy)
    affine_mat = (np.matrix(trans_mat) * np.matrix(rot_mat))[0:2, :]
    result = cv2.warpAffine(image, affine_mat, new_image_size, flags=cv2.INTER_LINEAR)

    return result

In [21]:
JSON_PATH = '/home/abert/Documents/NumeriCube/eurosilicone/gcaesthetics-implantbox/dataset/step3_orientationfixer/img_val/ann/'
OUTPUT_PATH = '/home/abert/Documents/NumeriCube/eurosilicone/gcaesthetics-implantbox/dataset/step3_orientationfixer/without_crop/img_val/img_corr_full/'
IMAGE_PATH = '/home/abert/Documents/NumeriCube/eurosilicone/gcaesthetics-implantbox/dataset/step3_orientationfixer/without_crop/img_val/img/'
filenames = os.listdir(IMAGE_PATH)
print(filenames)
for filename in filenames:
    
    print(filename)
    
    #Read JSON data
    if filename:
        with open(JSON_PATH + filename +'.json', 'r') as f:
            x = json.load(f)
    #Store the coordinates of the two points creating the vector
    x1 = int(x['objects'][0]['points']['exterior'][0][0])
    y1 = int(x['objects'][0]['points']['exterior'][0][1])
    x2 = int(x['objects'][0]['points']['exterior'][1][0])
    y2 = int(x['objects'][0]['points']['exterior'][1][1])
    
    
    size = np.zeros(2)
    size[0] = x['size']['width']
    size[1] = x['size']['height']
    print(size)
    # Removing the ".json" at the end to get the image name
    im_name = filename
    print(im_name)
    img2= cv2.imread(IMAGE_PATH+im_name,0)
    
    #Check points role ( is it the center point or the other ?)
    # The one closer to the real center of the image is the center point
    xcenter = size[0]/2
    ycenter = size[1]/2
    dist1 = np.sqrt((ycenter-y1)**2+(xcenter-x1)**2)
    dist2 = np.sqrt((ycenter-y2)**2+(xcenter-x2)**2)
    print(dist1,dist2)
    
    if dist1 > dist2:
        xc = x2
        yc = y2
        x_bord = x1
        y_bord = y1
        
    else:
        xc = x1
        yc = y1
        xb = x2
        yb = y2
    print(xc,yc)
    
    # Get the angle
    a = xb - xc
    b = yb - yc
    c = np.sqrt((b)**2+(a)**2)
    cos_alpha = a/c
    sin_alpha = b/c
    print('cos_alpha =', cos_alpha)
    print('sin_alpha =', sin_alpha)
    
    #alpha = math.acos(cos_alpha)

    if sin_alpha>=0:
        alpha = -1*(math.acos(cos_alpha) + np.pi/2)
    else:
        alpha =(math.acos(cos_alpha) - np.pi/2)
        
    #plt.imshow(img2)
    #plt.show()
    
    # Petits chagements pour que ça colle dans ce cas là
    #alpha = -1*(alpha+np.pi/2)
    print(math.degrees(alpha))
    
    #img3 = rotateImage(img2,-1*math.degrees(alpha))
    
    alpha_correc = int(-1*math.degrees(alpha))
    print(alpha_correc)
    img3 = imutils.rotate(img2,alpha_correc)
    print(type(img3))
    #plt.imshow(img3)
    #plt.show()
    print(img3.shape)
    
    #img4 =cv2.resize(img3, (224, 224))
    

    
    
    
    
    
    
    final = img3
    


    #final_im.save('./full_size/chip_test/'+im_name)
    cv2.imwrite( OUTPUT_PATH+im_name, final )

['FULL-2019-04-26-143540.png', 'FULL-2019-04-26-134003.png', 'FULL-2019-04-26-161730.png', 'FULL-2019-04-26-144550.png', 'Image__2019-05-16__14-12-25.png', 'FULL-2019-04-26-144532.png', 'FULL-2019-04-26-153019.png', 'FULL-2019-04-26-141551.png', 'FULL-2019-04-26-134625.png', 'FULL-2019-05-16-145900.png', 'FULL-2019-04-26-145730.png', 'FULL-2019-04-26-143453.png', 'FULL-2019-04-26-155943.png', 'FULL-2019-05-16-150629.png', 'FULL-2019-04-26-121510.png', 'FULL-2019-05-16-144716.png', 'FULL-2019-04-26-155747.png', 'FULL-2019-04-26-154958.png', 'FULL-2019-05-16-150402.png', 'FULL-2019-04-26-153252.png', 'FULL-2019-04-26-121658.png', 'FULL-2019-04-26-121709.png', 'FULL-2019-04-26-141007.png', 'FULL-2019-04-26-145244.png', 'FULL-2019-04-26-150809.png', 'FULL-2019-04-26-135349.png', 'FULL-2019-04-26-160919.png', 'FULL-2019-04-26-141623.png', 'FULL-2019-04-26-143515.png', 'FULL-2019-04-26-144514.png', 'FULL-2019-05-16-143907.png', 'FULL-2019-04-26-155805.png', 'FULL-2019-05-16-151030.png', 'FUL

92.13034245024816 653.9755347105884
900 775
cos_alpha = -0.9880891179202631
sin_alpha = 0.15388273147938522
-261.14799590061057
261
<class 'numpy.ndarray'>
(1634, 1636)
FULL-2019-04-26-144514.png
[1680. 1666.]
FULL-2019-04-26-144514.png
163.783393541592 555.7445456322536
700 748
cos_alpha = 0.2767067468870845
sin_alpha = 0.9609544090263423
-163.9362490112477
163
<class 'numpy.ndarray'>
(1666, 1680)
FULL-2019-05-16-143907.png
[1534. 1534.]
FULL-2019-05-16-143907.png
51.62363799656123 686.7503185292309
730 803
cos_alpha = 0.13136325097687
sin_alpha = 0.9913343009766119
-172.45162352718685
172
<class 'numpy.ndarray'>
(1534, 1534)
FULL-2019-04-26-155805.png
[1604. 1604.]
FULL-2019-04-26-155805.png
83.23460818673925 684.2404548110262
870 754
cos_alpha = -0.8883460977830477
sin_alpha = 0.45917448813455625
-242.666148449669
242
<class 'numpy.ndarray'>
(1604, 1604)
FULL-2019-05-16-151030.png
[1552. 1495.]
FULL-2019-05-16-151030.png
66.97947446792935 664.0152483188922
784 814
cos_alpha = -0.663